In [2]:

from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()


from azure.ai.ml import MLClient
ml_client = MLClient(
    credential=credential,
    subscription_id="8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5",
    resource_group_name="sparc2023-workspace-xudyu-rg",
    workspace_name="sparc2023-ws-xudyu",
)




In [3]:
from azure.ai.ml.entities import Environment
import os
project_dir="."
dependencies_dir = os.path.join(project_dir,"dependencies/")

pipeline_job_env = Environment(
    name="conda-DETR",
    description="env for DETR experiments",
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/curated/acpt-pytorch-2.0-cuda11.7:15"
    #set OS var
    
    )

env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name conda-DETR is registered to workspace, the environment version is 5


In [11]:
from azure.ai.ml import command
from azure.ai.ml import Input, Output
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes


outputs = {
    "output_logs": Output(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/logs/",
                        mode=InputOutputModes.RW_MOUNT
                        )
}
inputs={
        "input_data": Output(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/",
                        mode=InputOutputModes.RO_MOUNT
                        )
}

registered_model_name = "Contrastive trained PAIR Detr Model"
job = command(
    code=project_dir,  # location of source code
    command="python main.py --coco_path ${{inputs.input_data}}, --output_dir ${{outputs.output_logs}} --batch_size 2 ",    environment=pipeline_job_env.name+":"+pipeline_job_env.version,
    compute="smander",
    experiment_name="PairDETR single run",
    display_name="single pair DETR run stephenM",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
        "node_count": 1,
        "instance_count": 1,
    },
    inputs=inputs,
    outputs=outputs,
)

In [12]:
ml_client.create_or_update(job)

HttpResponseError: (UserError) Unable to create or update run crimson_caravan_6z2dqh7fd6 : Bad Request.
Code: UserError
Message: Unable to create or update run crimson_caravan_6z2dqh7fd6 : Bad Request.
Additional Information:Type: ComponentName
Info: {
    "value": "managementfrontend"
}Type: Correlation
Info: {
    "value": {
        "operation": "cab72bd9243e8395d584c86bc1408409",
        "request": "623e8650da292b4e"
    }
}Type: Environment
Info: {
    "value": "uksouth"
}Type: Location
Info: {
    "value": "uksouth"
}Type: Time
Info: {
    "value": "2023-10-23T14:34:22.7518241+00:00"
}Type: InnerError
Info: {
    "value": {
        "code": "BadArgument",
        "innerError": {
            "code": "CreatRunBadRequest",
            "innerError": null
        }
    }
}

In [4]:
#Lets define some sweeps, We really want to trial a load of values for all the following:
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential
''' 
    parser.add_argument('--lr', default=1e-4, type=float)
    parser.add_argument('--lr_backbone', default=1e-6, type=float)
    
    # Model parameters
    parser.add_argument('--frozen_weights', type=str, default=None,
                        help="Path to the pretrained model. If set, only the mask head will be trained")
    # * Backbone
    parser.add_argument('--backbone', default='resnet50', type=str,

    parser.add_argument('--enc_layers', default=4, type=int,
    parser.add_argument('--dec_layers', default=4, type=int,
    parser.add_argument('--dim_feedforward', default=512, type=int,
    parser.add_argument('--hidden_dim', default=512, type=int,
    parser.add_argument('--dropout', default=0.1, type=float,
    parser.add_argument('--nheads', default=16, type=int,
    parser.add_argument('--num_queries', default=1, type=int,

    # * Segmentation
    parser.add_argument('--masks', action='store_true',)
    # Loss
    parser.add_argument('--no_aux_loss', dest='aux_loss', action='store_false',default=True,
                        help="Disables auxiliary decoding losses (loss at each layer)")
    # * Matcher
    parser.add_argument('--set_cost_class', default=0.002, type=float,  
    parser.add_argument('--set_cost_bbox', default=5, type=float,
    parser.add_argument('--set_cost_giou', default=2, type=float,

    # * Loss coefficients
    parser.add_argument('--mask_loss_coef', default=10, type=float)
    parser.add_argument('--dice_loss_coef', default=1, type=float) # these should sum to one?
    parser.add_argument('--cls_loss_coef', default=1, type=float) # 1 /hdim
    parser.add_argument('--bbox_loss_coef', default=5, type=float)
    parser.add_argument('--giou_loss_coef', default=2, type=float)
    parser.add_argument('--focal_alpha', default=0.25, type=float)

    parser.add_argument('--loss_gt_iou', default=0.5, type=float)            #need to do sweep over these
    parser.add_argument('--loss_out_iou', default=0.5, type=float)         #need to do sweep over these     
    parser.add_argument('--class_loss_coef', default=1, type=float)        #need to do sweep over these
    parser.add_argument('--class_mask_loss_coef', default=2, type=float)   #need to do sweep over these

    parser.add_argument('--method', choice of "linear_sum","v1", "v2",...."v5"


    # dataset parameters
    parser.add_argument('--output_dir', default='',
                        help='path where to save, empty for no saving')

'''
from azure.ai.ml import command
from azure.ai.ml import Input, Output
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes


outputs = {
    "output_logs": Output(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/logs/",
                        mode=InputOutputModes.RW_MOUNT
                        )
}
inputs={
        #TO DO :: Should really be putting our data here as a path too 
        "batch_size":Choice(range(2,6)),
        "method": Choice(["linear_sum","v6", "v2", "v3", "v4", "v5","fastcriterion"]),
        "loss_gt_iou": Choice([0.2,0.5,0.75,1,2]),
        "loss_out_iou": Choice([0.2,0.5,0.75,1,2]),
        "class_loss_coef": Choice([0.2,0.5,0.75,1,2]),
        "class_mask_loss_coef": Choice([0.2,0.5,0.75,1,2]),
        "num_queries": Choice([1,2,3,4,5,6,7,8,9,10]),
        "input_data": Input(type=AssetTypes.URI_FOLDER,
                        path="azureml://subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu/datastores/workspaceblobstore/paths/data/",
                        mode=InputOutputModes.RO_MOUNT
                        )
}

# Create your command
command_job_for_sweep = command(
    code=project_dir,  # location of source code
    command="python main.py --coco_path ${{inputs.input_data}} --output_dir ${{outputs.logs}}",
    environment=pipeline_job_env.name+"@latest",
    compute="cpu-cluster",
    experiment_name="Testrelationshipdetection",
    display_name="DETR Sweep-StephenM",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
    },
    inputs=inputs,
    outputs=outputs,
)

# Call sweep() on your command job to sweep over your parameter expressions
sweep_job = command_job_for_sweep.sweep(
    compute="smander", 
    sampling_algorithm="random",
    primary_metric="train_loss",#should really set this to something at the validation stage 
    goal="Minimize",
)
# Define the limits for this sweep
sweep_job.set_limits(max_total_trials=500)

# Set early stopping on this one
sweep_job.early_termination = MedianStoppingPolicy(delay_evaluation=5, evaluation_interval=2)

# Specify your experiment details
sweep_job.display_name = "Train PairDETR"
sweep_job.experiment_name = "StephenM-efficeintLSA "
sweep_job.description = "Run a hyperparameter sweep job for assessing how LSA affects performance"

# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)

# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

# Download best trial model output
#ml_client.jobs.download(returned_sweep_job.name, output_name="model")



ValidationException: Unsupported input type: <class 'azure.ai.ml.entities._inputs_outputs.output.Output'>, only Input, dict, str, bool, int and float are supported.

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input

registered_model_name = "Find Masks from CLIP Model"
job = command(
    code=os.path.join(project_dir),  # location of source code
    command="python ClipToMask.py --Cache_dir {} --batch_size {}".format("./data",2) ,#--data ${{inputs.datadir}}",
    environment=pipeline_job_env.name+"@latest",
    compute="sparc-v100-low-priority",
    experiment_name="MaskPredictionwithCLIPVisGenome",
    display_name="MASKFINDER-STeveM",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
    },
)
ml_client.create_or_update(job)

Experiment,Name,Type,Status,Details Page
MaskPredictionwithCLIPVisGenome,coral_music_r2zb7flf8s,command,Starting,Link to Azure Machine Learning studio


In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy
from azure.identity import DefaultAzureCredential

# Create your command
command_job_for_sweep = command(
    code=project_dir,  # location of source code
    command="python ClipToMask.py",#--data ${{inputs.datadir}}",
    environment=pipeline_job_env.name+"@latest",
    compute="cpu-cluster",
    experiment_name="Test CLIP Learning of MASKS",
    display_name="SweepTest-StephenM",
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 1,
    },
    inputs={
        #TO DO :: Should really be putting our data here as a path too 
        "version":Choice([1,2]),
        "batch_size":Choice(range(1,2)),
        "layers":Choice([2,3,4])
    },
)

# Call sweep() on your command job to sweep over your parameter expressions
sweep_job = command_job_for_sweep.sweep(
    compute="sparc-v100-low-priority-hv", 
    sampling_algorithm="random",
    primary_metric="train_loss",#should really set this to something at the validation stage 
    goal="Minimize",
)
# Define the limits for this sweep
sweep_job.set_limits(max_total_trials=50, max_concurrent_trials=8, timeout=14400)

# Set early stopping on this one
sweep_job.early_termination = MedianStoppingPolicy(delay_evaluation=5, evaluation_interval=2)

# Specify your experiment details
sweep_job.display_name = "CLIP-MaskLearning-sweep"
sweep_job.experiment_name = "StephenM-CLIP-Mask-Finding"
sweep_job.description = "Run a hyperparameter sweep job for assessing how good MASK finding is from CLIP embeddings"

# submit the sweep
returned_sweep_job = ml_client.create_or_update(sweep_job)

# get a URL for the status of the job
returned_sweep_job.services["Studio"].endpoint

# Download best trial model output
#ml_client.jobs.download(returned_sweep_job.name, output_name="model")


'https://ml.azure.com/runs/quiet_spoon_1r3pz4w1p8?wsid=/subscriptions/8db6e466-5fa0-4e7f-b009-c5e20e1a7fe5/resourcegroups/sparc2023-workspace-xudyu-rg/workspaces/sparc2023-ws-xudyu&tid=c681f89a-795a-4473-bc07-d86cb09d4312'